<a href="https://colab.research.google.com/github/SIDIBEMoussa/NLP-Trainings/blob/main/03_Exercice_LM_classification_and_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import pandas as pd

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [38]:
import pathlib, os

In [39]:
path = "/content/datasets/lda_sports_politics_docs"

file_directory = pathlib.Path(path).exists()

if not file_directory:

    !git clone https://github.com/selva86/datasets.git



text_files = os.listdir(path)

In [40]:
text_files

['shinzo abe.txt',
 'pizza.txt',
 'lee quan yew.txt',
 'queen elizabeth.txt',
 'cricket.txt',
 'idli.txt',
 'barack obama.txt',
 'baseball.txt',
 'badminton.txt',
 'noodles.txt',
 'tipu sultan.txt',
 'table tennis.txt',
 'pasta.txt',
 'narendra modi.txt',
 'dosa.txt']

In [41]:
articles=[]

for filename in text_files:

    article = pathlib.Path(path+'/'+filename).read_text()

    articles.append(article)

In [42]:
articles[0]

"Shinzō Abe (Abe Shinzō, IPA: born 21 September 1954) is the current Prime Minister of Japan, re-elected to the position in December 2012. Abe is also the President of the Liberal Democratic Party (LDP).\nAbe served for a year as Prime Minister, from 2006 to 2007. Hailing from a politically prominent family, at age 52, Abe became Japan's youngest post-war prime minister, and the first to be born after World War II, when he was elected by a special session of the National Diet in September 2006. Abe resigned on 12 September 2007, for health reasons. Abe was replaced by Yasuo Fukuda, beginning a string of five Prime Ministers, none of whom retained office for more than sixteen months, before Abe staged a political comeback.\nOn 26 September 2012, Abe defeated former Minister of Defense Shigeru Ishiba, in a run-off vote, to win the LDP Presidential Election. Following the LDP's landslide victory in the 2012 general election, Abe became the Prime Minister again. Abe is the first former Pri

In [43]:
from pprint import pprint # for printing objects nicely

from gensim import corpora, models
from gensim.utils import simple_preprocess

# from gensim.parsing.preprocessing import STOPWORDS 
# Instead of the gensim english stopwords, we use nltk's German stopwords
from nltk.corpus import stopwords

from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import *

import numpy as np

from random import choice

np.random.seed(1234)

stemmer = SnowballStemmer('english')
german_stop_words = set(stopwords.words('english'))


def lemmatize_stemming(text):
  return stemmer.stem(text)

def preprocess(text):
  result = [lemmatize_stemming(token) 
            for token in simple_preprocess(text)  
            if token not in german_stop_words and len(token) > 3]
  return result

In [44]:
print('original document: ')
article = choice(articles)
print(article)

# This time, we don't care about punctuations as tokens (Can you think why?):
print('original document, broken into words: ')
words = [word for word in article.split(' ')]
print(words)
print("Vocabulary size of the original article:", len(set(words)))

# now let's see what happens when we pass the article into our preprocessing 
# method:
print('\n\n tokenized and lemmatized document: ')
preprocessed_article = preprocess(article)
print(preprocessed_article)
print("Vocabulary size after preprocessing:", len(set(preprocessed_article)))

original document: 
Pasta is a staple food of traditional Italian cuisine, with the first reference dating to 1154 in Sicily. It is also commonly used to refer to the variety of pasta dishes. Typically, pasta is a noodle made from an unleavened dough of a durum wheat flour mixed with water or eggs and formed into sheets or various shapes, then cooked by boiling or baking. It can also be made with flour from other cereals or grains. Pastas may be divided into two broad categories, dried (pasta secca) and fresh (pasta fresca).
Most dried pasta is commercially produced via an extrusion process. Fresh pasta was traditionally produced by hand, sometimes with the aid of simple machines, but today many varieties of fresh pasta are also commercially produced by large-scale machines, and the products are widely available in supermarkets.
Both dried and fresh pasta come in a number of shapes and varieties, with 310 specific forms known variably by over 1300 names having been documented. In Italy

In [45]:
processed_docs = list(map(preprocess, articles))
processed_docs[:2]

[['shinzō',
  'shinzō',
  'born',
  'septemb',
  'current',
  'prime',
  'minist',
  'japan',
  'elect',
  'posit',
  'decemb',
  'also',
  'presid',
  'liber',
  'democrat',
  'parti',
  'serv',
  'year',
  'prime',
  'minist',
  'hail',
  'polit',
  'promin',
  'famili',
  'becam',
  'japan',
  'youngest',
  'post',
  'prime',
  'minist',
  'first',
  'born',
  'world',
  'elect',
  'special',
  'session',
  'nation',
  'diet',
  'septemb',
  'resign',
  'septemb',
  'health',
  'reason',
  'replac',
  'yasuo',
  'fukuda',
  'begin',
  'string',
  'five',
  'prime',
  'minist',
  'none',
  'retain',
  'offic',
  'sixteen',
  'month',
  'stage',
  'polit',
  'comeback',
  'septemb',
  'defeat',
  'former',
  'minist',
  'defens',
  'shigeru',
  'ishiba',
  'vote',
  'presidenti',
  'elect',
  'follow',
  'landslid',
  'victori',
  'general',
  'elect',
  'becam',
  'prime',
  'minist',
  'first',
  'former',
  'prime',
  'minist',
  'return',
  'offic',
  'sinc',
  'shigeru',
  'yoshi

In [46]:
dictionary = corpora.Dictionary(processed_docs)

In [47]:
for idx, (k, v) in enumerate(dictionary.iteritems()):
    print(k, v)
    if idx >= 10:
        break

0 also
1 becam
2 begin
3 born
4 coalit
5 comeback
6 current
7 decemb
8 defeat
9 defens
10 democrat


### Model Hyperparameter tuning

In [48]:
## Model hyper parameters:

## These are the dictionary preparation parameters:
filter_tokens_if_container_documents_are_less_than = 1
filter_tokens_if_appeared_percentage_more_than = 0.5
keep_the_first_n_tokens=100000

## and the LDA Parameters: 
num_of_topics = 10

In [49]:
dictionary.filter_extremes(
        no_below=filter_tokens_if_container_documents_are_less_than, 
        no_above=filter_tokens_if_appeared_percentage_more_than, 
        keep_n=keep_the_first_n_tokens)

In [ ]:
def optimization(min_topic=4, max_topic=10,more_than_number=10,less_than_range=20):

    perplexity = {}
    
    for num_topic in range(min_topic,max_topic+1):

      for more_than in np.linspace(0,1,more_than_number):

        for less_than in range(less_than_range):

            dictionary.filter_extremes(
            no_below=less_than, 
            no_above=more_than, 
            keep_n=keep_the_first_n_tokens)

            bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

            lda_model = models.LdaMulticore(bow_corpus, 
                                  num_topics=num_topic, 
                                  id2word=dictionary, 
                                  passes=10, 
                                  workers=2)

In [58]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

bow_corpus[:1]

[[(0, 1),
  (1, 2),
  (2, 1),
  (3, 2),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 6),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 2),
  (17, 1),
  (18, 2),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 2),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 7),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 2),
  (32, 1),
  (33, 1),
  (34, 2),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 6),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 2),
  (45, 1),
  (46, 4),
  (47, 1),
  (48, 1),
  (49, 2),
  (50, 2),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1)]]

In [52]:
# randomly choose an article from the corpus:
sample_bow_doc = choice(bow_corpus)

print('The processed bag-of-word document is just pairs of (word_id, # of occurnces) and looks like this:')
print(sample_bow_doc, '\n\n')

print ('We peek in the dictionary: for each word_id, we get its assigned word:')
for word_id, word_freq in sample_bow_doc:
  real_word = dictionary[word_id]
  print(f'Word {word_id} ("{real_word}") appears {word_freq} time.')

The processed bag-of-word document is just pairs of (word_id, # of occurnces) and looks like this:
[(0, 1), (1, 1), (7, 1), (57, 1), (103, 1), (129, 1), (130, 1), (150, 1), (155, 4), (167, 1), (168, 1), (242, 2), (244, 1), (286, 1), (289, 1), (392, 1), (413, 1), (422, 2), (425, 1), (439, 1), (501, 1), (502, 1), (503, 2), (504, 2), (505, 1), (506, 1), (507, 1), (508, 1), (509, 1), (510, 3), (511, 1), (512, 1), (513, 1), (514, 1), (515, 1), (516, 1), (517, 1), (518, 1), (519, 1), (520, 1), (521, 1), (522, 1), (523, 1), (524, 1), (525, 1), (526, 1), (527, 1), (528, 1), (529, 1), (530, 1), (531, 3), (532, 1), (533, 1), (534, 2), (535, 1), (536, 1), (537, 1), (538, 1), (539, 1), (540, 1), (541, 1), (542, 1), (543, 1), (544, 1), (545, 1), (546, 1), (547, 1), (548, 1), (549, 1), (550, 1), (551, 1), (552, 8), (553, 1), (554, 1), (555, 1), (556, 1), (557, 2), (558, 1), (559, 1), (560, 3), (561, 1), (562, 1), (563, 2), (564, 1), (565, 1), (566, 1), (567, 1), (568, 1), (569, 1), (570, 1), (571, 1

## LDA model using Bag-of-words

In [54]:
lda_model = models.LdaMulticore(bow_corpus, 
                                num_topics=num_of_topics, 
                                id2word=dictionary, 
                                passes=10, 
                                workers=2)

/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [20]:
for idx, topic in lda_model.print_topics(num_of_topics):
    print(f'Topic: {idx} \t Words: {topic}')

Topic: 0 	 Words: 0.028*"ball" + 0.028*"tabl" + 0.021*"player" + 0.014*"rule" + 0.014*"must" + 0.014*"play" + 0.014*"side" + 0.014*"return" + 0.014*"bounc" + 0.008*"also"
Topic: 1 	 Words: 0.057*"dosa" + 0.017*"origin" + 0.017*"udupi" + 0.017*"present" + 0.017*"karnataka" + 0.017*"popular" + 0.012*"made" + 0.012*"accord" + 0.012*"tamil" + 0.012*"centuri"
Topic: 2 	 Words: 0.023*"minist" + 0.018*"prime" + 0.016*"elect" + 0.013*"play" + 0.012*"born" + 0.011*"septemb" + 0.009*"elizabeth" + 0.009*"world" + 0.009*"serv" + 0.009*"team"
Topic: 3 	 Words: 0.036*"noodl" + 0.016*"minist" + 0.016*"modi" + 0.016*"gujarat" + 0.012*"sinc" + 0.012*"common" + 0.012*"improv" + 0.012*"serv" + 0.012*"word" + 0.012*"mani"
Topic: 4 	 Words: 0.047*"tipu" + 0.038*"mysor" + 0.019*"father" + 0.019*"sultan" + 0.014*"rocket" + 0.014*"british" + 0.014*"french" + 0.010*"struggl" + 0.010*"ruler" + 0.010*"kingdom"
Topic: 5 	 Words: 0.034*"team" + 0.031*"bat" + 0.031*"run" + 0.026*"score" + 0.024*"ball" + 0.021*"fiel

## TF / IDF

In [21]:
# initialize a tfidf from our corpus
tfidf = models.TfidfModel(bow_corpus)

# apply it on our corpus 
tfidf_corpus = tfidf[bow_corpus]

pprint(tfidf_corpus[0][:10])

[(0, 0.03278411436243488),
 (1, 0.09458263214900205),
 (2, 0.09689176633988147),
 (3, 0.07861485368951274),
 (4, 0.09689176633988147),
 (5, 0.09689176633988147),
 (6, 0.0575843394951251),
 (7, 0.07209154120719125),
 (8, 0.07209154120719125),
 (9, 0.09689176633988147)]


In [22]:
# the new tfidf corpus is just our corpus - but transformed. It has the same size of documents:
assert len(bow_corpus) == len(tfidf_corpus)

Now let's apply LDA on the tfidf corpus, with the same amount of topics.

You can play with the # of passes, if the model doesn't converge properly

In [24]:
lda_model_tfidf = models.LdaMulticore(tfidf_corpus, 
                                      num_topics=num_of_topics, 
                                      id2word=dictionary, 
                                      passes=10, 
                                      workers=4)

In [25]:
for idx, topic in lda_model_tfidf.print_topics(num_of_topics):
    print(f'Topic: {idx} \t Word: {topic}')

Topic: 0 	 Word: 0.008*"pizza" + 0.006*"ball" + 0.006*"tabl" + 0.005*"player" + 0.005*"plate" + 0.005*"base" + 0.004*"around" + 0.004*"home" + 0.004*"run" + 0.004*"bat"
Topic: 1 	 Word: 0.009*"dosa" + 0.008*"noodl" + 0.005*"popular" + 0.005*"id" + 0.004*"cake" + 0.004*"word" + 0.004*"ferment" + 0.003*"made" + 0.003*"origin" + 0.003*"variant"
Topic: 2 	 Word: 0.001*"serv" + 0.001*"also" + 0.001*"mani" + 0.001*"rule" + 0.001*"food" + 0.001*"team" + 0.001*"minist" + 0.001*"take" + 0.001*"world" + 0.001*"known"
Topic: 3 	 Word: 0.004*"tanjong" + 0.004*"kuan" + 0.004*"pagar" + 0.004*"minist" + 0.003*"member" + 0.003*"year" + 0.003*"prime" + 0.003*"harri" + 0.003*"spmj" + 0.003*"star"
Topic: 4 	 Word: 0.001*"also" + 0.001*"serv" + 0.001*"mani" + 0.001*"team" + 0.001*"dish" + 0.001*"food" + 0.001*"minist" + 0.001*"world" + 0.001*"sinc" + 0.001*"becam"
Topic: 5 	 Word: 0.001*"serv" + 0.001*"also" + 0.001*"team" + 0.001*"mani" + 0.001*"food" + 0.001*"world" + 0.001*"known" + 0.001*"side" + 0.00

## Inference

Now that we have a topic-modeler, let's use it on one of the articles.

In [36]:
# randomly pick an article:
test_doc = choice(range(len(processed_docs)))
processed_docs[test_doc][:10]

['pasta',
 'stapl',
 'food',
 'tradit',
 'italian',
 'cuisin',
 'first',
 'refer',
 'date',
 'sicili']

Using the original BOW model:

In [27]:
for index, score in sorted(lda_model[bow_corpus[test_doc]], key=lambda tup: -1*tup[1]):
    print(f"Topic match score: {score} \nTopic: {lda_model.print_topic(index, num_of_topics)}")


Topic match score: 0.9936169385910034 
Topic: 0.047*"tipu" + 0.038*"mysor" + 0.019*"father" + 0.019*"sultan" + 0.014*"rocket" + 0.014*"british" + 0.014*"french" + 0.010*"struggl" + 0.010*"ruler" + 0.010*"kingdom"


And with the TF/IDF model:

In [28]:
for index, score in sorted(lda_model_tfidf[bow_corpus[test_doc]], key=lambda tup: -1*tup[1]):
    print("Topic match score: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, num_of_topics)))

Topic match score: 0.9936166405677795	 
Topic: 0.007*"pasta" + 0.006*"tipu" + 0.005*"mysor" + 0.005*"racquet" + 0.005*"minist" + 0.004*"elect" + 0.004*"shape" + 0.004*"prime" + 0.004*"side" + 0.004*"unit"


In [29]:
print('Perplexity: ', lda_model.log_perplexity(bow_corpus)) 
print('Perplexity TFIDF: ', lda_model_tfidf.log_perplexity(bow_corpus)) 

Perplexity:  -6.678647591068424
Perplexity TFIDF:  -9.63845659098846


### Exercise - inference

Now try it on a new document!

Go to a news website, such as [orf.at](https://orf.at/) and copy an article of your choice here:

In [30]:
#https://www.lemonde.fr/en/opinion/article/2022/12/27/speaking-the-truth-about-immigration_6009281_23.html

unseen_document = """You can share an article by clicking on the share icons at the top right of it. 
The total or partial reproduction of an article, without the prior written authorization of Le Monde, is strictly forbidden. 
For more information, see our Terms and Conditions. 
For all authorization requests, contact droitsdauteur@lemonde.fr. 

The French reversal may have only a remote connection to migration issues. In each of the target countries,
visa restrictions fueled strong criticism of France. But in a context of rising anti-French sentiment throughout
Africa and faced with the risk of emerging countries leaving the Western bloc, Paris was able to give precedence
to these fundamental concerns over the math of deportations. Other bilateral issues (Western Sahara, gas, etc.),
as well as the difficulty of giving in to Morocco without doing the same for Algeria, may also have played roles.
The equation cannot be summed up as a simple trade-off between visas and deportations, because relations between 
France and its former colonies, marked by the presence of large communities of Maghrebi origin, are a matter of 
both foreign policy and domestic affairs."""

bow_vector = dictionary.doc2bow(preprocess(unseen_document))

print("Simply printing the lda_model output would look like this:")
pprint(lda_model[bow_vector])

print("\n\nSo let's make it nicer, by printing the topic contents:")
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))


Simply printing the lda_model output would look like this:
[(0, 0.09516191),
 (1, 0.14162324),
 (2, 0.23643194),
 (3, 0.1383363),
 (4, 0.10559635),
 (6, 0.15127546),
 (9, 0.12003621)]


So let's make it nicer, by printing the topic contents:
Score: 0.23641768097877502	 Topic: 0.023*"minist" + 0.018*"prime" + 0.016*"elect" + 0.013*"play" + 0.012*"born"
Score: 0.15127290785312653	 Topic: 0.016*"democrat" + 0.016*"obama" + 0.016*"state" + 0.016*"presid" + 0.016*"primari"
Score: 0.14163684844970703	 Topic: 0.057*"dosa" + 0.017*"origin" + 0.017*"udupi" + 0.017*"present" + 0.017*"karnataka"
Score: 0.13833248615264893	 Topic: 0.036*"noodl" + 0.016*"minist" + 0.016*"modi" + 0.016*"gujarat" + 0.012*"sinc"
Score: 0.12004640698432922	 Topic: 0.040*"pasta" + 0.027*"shape" + 0.027*"pizza" + 0.017*"common" + 0.017*"fresh"
Score: 0.10559042543172836	 Topic: 0.047*"tipu" + 0.038*"mysor" + 0.019*"father" + 0.019*"sultan" + 0.014*"rocket"
Score: 0.09516466408967972	 Topic: 0.028*"ball" + 0.028*"tabl" + 

## Visualization

In [32]:
! pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 27.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=03b7274780823a65f3805d126014f05b1ea9d3411f6e7d69bd0bfd25c90e1f1d
  Stored in directory: /root/.cache/pip/wheels/90/61/ec/9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=deb781396c95b28f9bef1bf33365dce530c8deb7a94c598b57bf94afad6205c5
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built pyLDAvis sklearn


In [33]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

bow_lda_data = gensimvis.prepare(lda_model, bow_corpus, dictionary)

pyLDAvis.display(bow_lda_data)

/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable
/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
